# Economic calendar against S&P500 index observation

This code utilizing python package 'investpy' to pull out economic calendar data & S&P500 index price data from investing.com. <br>The objective is to relate percentage change price of index against economic calendar.

In [ ]:
# Install investpy
#!pip install investpy

In [44]:
import investpy as inv
import pandas as pd

# Set any float number precision 2 decimal points
pd.set_option('precision', 2)

In [45]:
# Define function for style
def above_zero(val):
  color = 'green' if val > 0 else 'red'
  return 'color: %s' % color

In [46]:
# Pull out index price data.
# TAKE NOTE that the date format is in dd/mm/yyyy.

df = inv.get_index_historical_data(index='S&P 500',
                                   country='United States',
                                   from_date='30/09/2021',
                                   to_date='23/10/2021')

# Copy file to prevent A value is trying to be set on a copy of a slice from a DataFrame
df = df.copy()

# Find Close price percentage
df.loc[:,'SP500_pct'] = df.loc[:,'Close'].pct_change()*100

In [47]:
# Pull out economic calendar data with 'high' importance.
# TAKE NOTE that the date format is in dd/mm/yyyy.

cal_2021 = inv.economic_calendar(from_date="30/09/2021", 
                                 to_date="23/10/2021")
df2 = cal_2021.query("importance == 'high'")

# Filter economic data only for currency USD, zone united states.
df2 = df2[df2['currency'] == 'USD']

In [48]:
# Copy file to prevent A value is trying to be set on a copy of a slice from a DataFrame.
df2 = df2.copy()

# Set 'date' object as date format.
df2.loc[:,'date'] = pd.to_datetime(df2['date'], format='%d/%m/%Y')

# Set 'date' as index.
df2.set_index('date',inplace = True)

In [50]:
# Filter dataframe (df) only on particular date based upon other dataframe (df2)
df3 = df[df.index.isin(df2.index.drop_duplicates(keep="first"))][['SP500_pct']]

In [73]:
# Combine data from index price (df3) onto economic data (df2) and filter only for month October
# The process use join command instead of merge since there's duplicate data from df2 (non-unique index).
df5 = df2.join(df3, how='outer').query("index.dt.month > 9")

# Reset the index to highlight row based on index name and apply style 'above zero'
df5.reset_index(drop=False).style.apply(lambda x: ['background: lightgreen'
                                                   if x.name in [10,11]
                                                   else '' for i in x], axis=1)\
                                 .applymap(above_zero, subset=['SP500_pct'])

,index,id,time,zone,currency,importance,event,actual,forecast,previous,SP500_pct
0,2021-10-01 00:00:00,436769,22:00,united states,USD,high,ISM Manufacturing PMI (Sep),61.1,59.6,59.9,1.15
1,2021-10-05 00:00:00,436943,22:00,united states,USD,high,ISM Non-Manufacturing PMI (Sep),61.9,60.0,61.7,1.05
2,2021-10-06 00:00:00,436977,20:15,united states,USD,high,ADP Nonfarm Employment Change (Sep),568K,428K,340K,0.41
3,2021-10-06 00:00:00,437988,22:30,united states,USD,high,Crude Oil Inventories,2.346M,-0.418M,4.578M,0.41
4,2021-10-07 00:00:00,438024,20:30,united states,USD,high,Initial Jobless Claims,326K,348K,364K,0.83
5,2021-10-08 00:00:00,437119,20:30,united states,USD,high,Nonfarm Payrolls (Sep),194K,500K,366K,-0.19
6,2021-10-08 00:00:00,437125,20:30,united states,USD,high,Unemployment Rate (Sep),4.8%,5.1%,5.2%,-0.19
7,2021-10-12 00:00:00,437292,22:00,united states,USD,high,JOLTs Job Openings (Aug),10.439M,10.925M,11.098M,-0.24
8,2021-10-13 00:00:00,437343,20:30,united states,USD,high,Core CPI (MoM) (Sep),0.2%,0.2%,0.1%,0.30
9,2021-10-14 00:00:00,438452,02:00,united states,USD,high,FOMC Meeting Minutes,None,None,None,1.71


<div class="alert alert-block alert-success">
On mid October 2021, index S&P500 had shown positive.<br>
News on October 14th, 2021 said: <font color='black'>"The S&P 500 jumped by 1.7% in its best day since March 2021."</font><br>
There are two macro drives that might help the sentiment:<br>
a. Initial Jobless Claims <font color='blue'>lower than expected</font>.<br>
b. Producer price index rise <font color='blue'>slower than previous month (at 0.5%)</font>.
</div>

<div class="alert alert-block alert-info">
Reference:<br/> 
1. https://github.com/alvarobartt/investpy<br>
2. https://medium.com/geekculture/a-simple-way-to-download-financial-data-from-investing-com-in-python-8262271c804f<br>
3. https://github.com/erykml/medium_articles/blob/master/Quantitative%20Finance/investpy.ipynb
</div>